In [ ]:
import os
from dotenv import load_dotenv
from groq import Groq
from IPython.display import display, Markdown, update_display

# --------------------------------------------------
# Load environment variables
# --------------------------------------------------
load_dotenv()

# Make sure GROQ_API_KEY is set
if not os.getenv("GROQ_API_KEY"):
    raise ValueError("❌ GROQ_API_KEY not found in environment variables")

# Initialize Groq client
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# --------------------------------------------------
# System Prompt
# --------------------------------------------------
system_prompt = """
You are a helpful, professional coding assistant and technical mentor.

IMPORTANT:
- Always connect new answers to topics discussed earlier in the conversation.
- Build upon previously explained concepts and reuse terminology already introduced.
- When a new concept appears, relate it to prior knowledge whenever possible.

The user will provide source code. Your responsibilities are:
1. Carefully read and fully understand the provided code.
2. Explain the code line by line using clear, beginner-friendly language.
3. Identify bugs, logical issues, or bad practices.
4. Provide corrected code when issues are found.
5. Suggest improvements for readability, modularity, and maintainability.
6. Follow industry best practices and explain WHY changes are useful.

Guidelines:
- Be precise and structured.
- Use clean formatting and code blocks.
- Prioritize learning over verbosity.
"""

# --------------------------------------------------
# State
# --------------------------------------------------
conversation_history = []
question_count = 0

# --------------------------------------------------
# Welcome Message
# --------------------------------------------------
print("🚀 Welcome to your AI Code Assistant!")
print("📌 Paste code, ask questions, and improve step by step.")
print("❌ Type 'quit', 'bye', or 'exit' anytime to leave.\n")

# --------------------------------------------------
# Main Loop
# --------------------------------------------------
while True:

    # Input handling
    if question_count == 0:
        user_input = input("📄 Please copy-paste your code here:\n")
    else:
        user_input = input(f"\n❓ Question {question_count + 1}: What do you want to do next?\n")

    # Exit condition
    if user_input.lower() in ["quit", "bye", "exit"]:
        print("\n👋 Thanks for using the AI Code Assistant. Happy coding!")
        break

    # Empty input check
    if not user_input.strip():
        print("⚠️ Please enter something.")
        continue

    # Store user message
    conversation_history.append({"role": "user", "content": user_input})
    question_count += 1

    print(f"\n🧠 Thinking about:\n{user_input}\n")

    # --------------------------------------------------
    # Groq Streaming Call
    # --------------------------------------------------
    stream = client.chat.completions.create(
        model="llama-3.1-8b-instant",   # ✅ FREE & BEST for coding
        messages=[
            {"role": "system", "content": system_prompt}
        ] + conversation_history,
        stream=True
    )

    # Display streaming response
    response = ""
    display_handle = display(Markdown(""), display_id=True)

    for chunk in stream:
        delta = chunk.choices[0].delta.content
        if delta:
            response += delta
            update_display(Markdown(response), display_id=display_handle.display_id)

    # Save assistant response
    conversation_history.append({"role": "assistant", "content": response})
